In [ ]:
import pandas as pd
df1=pd.read_csv('/content/drive/MyDrive/Extracted_Data.csv')
df1.head(5)

,Unnamed: 0,Original_position,findings,impressions
0,0,0,"there is no acute intracranial hemorrhage, mas...",1. no acute intracranial hemorrhage or mass ef...
1,1,1,"ap, lateral, and oblique views of the left foo...",comminuted intra-articular fracture proximal p...
2,2,2,the procedure was explained to the patient. a ...,uncomplicated ultrasound and fluoroscopically ...
3,3,3,the ventricles and extra-axial spaces are norm...,3-mm lesion within the left cerebellar hemisph...
4,4,4,"there is no evidence of fracture, dislocation,...",no evidence of fracture or subluxation.


In [ ]:
x=list(df1.columns)
x.remove('findings')
x.remove('impressions')
df2=df1.drop(x,axis=1)
df2

In [ ]:
print(df2.isna().sum())
df2.dropna(axis=0,inplace=True)
print(df2.isna().sum())

findings       0
impressions    0
dtype: int64
findings       0
impressions    0
dtype: int64


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import regex as re
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
def clean_data(list1,name,new_list1):
    for i in list1:
        i=str(i)
        new_string = re.sub(r'[^\w\s]', '', i)
        pattern = r'[0-9]'
        new_string1 = re.sub(pattern, '', new_string)
        new_string2 = re.sub(r'\([^)]*\)', '', new_string1)
        new_string1=new_string2.lower()
        if name=='findings':
            tokens = [w for w in new_string1.split() if not w in stop_words]
            tokens=' '.join(tokens)
            new_list1.append(tokens)
        elif name=='impressions':
            new_list1.append(new_string1)
    return new_list1
x1=[]
list1=clean_data(list(df2['findings']),df2['findings'].name,x1)

In [ ]:
x2=[]
list2=clean_data(list(df2['impressions']),df2['impressions'].name,x2)

In [ ]:
def clean_further(list1):
    list5=[]
    for i in list1:
        i=str(i)
        list1=i.split(' ')
        for i in range(list1.count('')):
            list1.remove('')
        x1=' '.join(list1)
        list5.append(x1)
    return list5
list6=clean_further(list1)

In [ ]:
list7=clean_further(list2)

In [ ]:
df3=pd.DataFrame({'findings':list6,
                'impression':list7})
df3

In [ ]:
df3['impression'] = df3['impression'].apply(lambda x : '_START_ '+ x + ' _END_')

In [ ]:
import random as rnd
def get_any_data(times,which):
    for i in range(times):
        x1=rnd.randint(0,len(df3['findings']))
        print(x1)
        print(df3[which][x1])
        print(df3['impression'][x1])
        print('--------')
get_any_data(3,'findings')

In [ ]:
import matplotlib.pyplot as plt
text_word_count = []
summary_word_count = []
# populate the lists with sentence lengths
for i in df3['findings']:
      text_word_count.append(len(i.split()))
for i in df3['impression']:
      summary_word_count.append(len(i.split()))
length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})
length_df.hist(bins = 30)
plt.show()

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df3['findings'],df3['impression'],test_size=0.1,random_state=42,shuffle=True)

In [ ]:
X_train.shape

(2568,)

In [ ]:
list_len=[len(i.split()) for i in df3['impression']]
print(max(list_len))

209


In [ ]:
max_len_text=200
max_len_summary=190
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
in_tokenizer = Tokenizer()
in_tokenizer.fit_on_texts(X_train)
tr_tokenizer = Tokenizer()
tr_tokenizer.fit_on_texts(y_train)
X_train= in_tokenizer.texts_to_sequences(X_train) 
y_train= tr_tokenizer.texts_to_sequences(y_train)

In [ ]:
en_in_data= pad_sequences(X_train,  maxlen=max_len_text, padding='post') 
dec_data= pad_sequences(y_train,  maxlen=max_len_summary, padding='post')
dec_in_data = dec_data[:,:-1]
dec_tr_data = dec_data.reshape(len(dec_data),max_len_summary,1)[:,1:]

In [ ]:
x_voc_size   =  len(in_tokenizer.word_index) +1
y_voc_size   =  len(tr_tokenizer.word_index) +1

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

In [ ]:
size_of_vocab=len(in_tokenizer.word_index)+1

So this the technique of using embedding matrix and embedding vectors as embedding layers

In [ ]:
import numpy as np
#embedding Layer creation
from gensim.models import Word2Vec
best_model=Word2Vec.load('/content/drive/MyDrive/word2vec+fastText/word2vec.model')
vocab_len_size = best_model.wv.index2word
dict1={}
for i in range(len(best_model.wv.index2word)):
  keys=list(best_model.wv.vocab.keys())[i]
  vector1=best_model.wv.vectors[i]
  dict1[f'{keys}']=vector1
with open('/content/drive/MyDrive/custom_embeddings_word2vec.txt', 'w+') as f:
    for token, vector in dict1.items():
        vector_str = ' '.join([str(v) for v in vector])
        f.write(f'{token} {vector_str}\n')
embeddings_index = dict()
f = open('/content/drive/MyDrive/custom_embeddings_word2vec.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
x_voc_size=44082
print('Loaded %s word vectors.' % len(embeddings_index))
from keras.layers import Embedding
embedding_matrix = np.zeros((x_voc_size, 100))

#..so here the x_voc_size need to be replace by 44082 and 130 need to be replaced with 100

for word, i in in_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(dict1)+1,
                            130,
                            weights=[embedding_matrix],
                            input_length=max_len_text,
                            trainable=False)
#print(embedding_matrix.shape)
'''list_embeddings=list(dict1.keys())
if 'endotracheal' in list_embeddings:
  print(True)
else:
  print(False)'''
embedding_layer=Embedding(x_voc_size,100,weights=[embedding_matrix],input_length=max_len_text,trainable=False)
print(embedding_matrix[3].shape)

Loaded 44082 word vectors.
(100,)


In [ ]:
len(embedding_matrix)

44082

In [ ]:
#import json
#with open('/content/drive/MyDrive/data.json') as json_file:
 #   data = json.load(json_file)
  #  print(data)

In [ ]:
#list1=list(data.values())
#print(list1)

Using Clinical Bert Embeddings.Just Clinical Bert.
Next target is to use Clinical Biobert as your embedding matrix

In [ ]:
import tensorflow
import tensorflow.keras.layers
import tensorflow.keras.models
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import Model
from tensorflow import keras
from keras import layers
from keras import backend as K
from tensorflow.keras.layers import Attention

K.clear_session() 
latent_dim = 100
 
#create input object of total number of encoder words
#en_inputs = Input(shape=(max_len_text,)) 
#en_embedding = Embedding(x_voc_size, latent_dim,trainable=True)(en_inputs) 

#... adding a custom layer in place of this embedding layer for custom embedding scenario
sequence_input = Input(shape=(max_len_text,), dtype='int32')
en_embedding = embedding_layer(sequence_input)


#1st LSTM Layer
en_lstm1= LSTM(latent_dim, return_state=True, return_sequences=True) 
en_outputs1, state_h1, state_c1= en_lstm1(en_embedding) 
#2nd LSTM Layer
en_lstm2= LSTM(latent_dim, return_state=True, return_sequences=True) 
en_outputs2, state_h2, state_c2= en_lstm2(en_outputs1) 
#3rd LSTM Layer
en_lstm3= LSTM(latent_dim,return_sequences=True,return_state=True)
en_outputs3 , state_h3 , state_c3= en_lstm3(en_outputs2)
#encoder states
en_states= [state_h3, state_c3]

#---------------------------------


# Decoder. 
'''dec_inputs = Input(shape=(None,)) 

dec_emb_layer = Embedding(y_voc_size, latent_dim) 
dec_embedding = dec_emb_layer(dec_inputs) '''

#.using the custom decoder Layer
#................................................

dec_inputs = Input(shape=(None,), dtype='int32')
dec_embedding = embedding_layer(dec_inputs)
 
#initialize decoder's LSTM layer with the output states of encoder
dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
dec_outputs, *_ = dec_lstm(dec_embedding,initial_state=en_states) 

'''# Decoder. 
dec_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_voc_size, latent_dim) 
dec_embedding = dec_emb_layer(dec_inputs) 
 
#initialize decoder's LSTM layer with the output states of encoder
dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
dec_outputs, *_ = dec_lstm(dec_embedding,initial_state=en_states)'''

#Attention layer
attention =Attention()
attn_out = attention([dec_outputs,en_outputs3])
 
#Concatenate the attention output with the decoder outputs
merge=Concatenate(axis=-1, name='concat_layer1')([dec_outputs,attn_out]) 
#Dense layer (output layer)
dec_dense = Dense(x_voc_size, activation='softmax') 
dec_outputs = dec_dense(merge) 

model = Model([sequence_input, dec_inputs],dec_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        multiple             4408200     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 200, 100),   80400       ['embedding_1[0][0]']            
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                              

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
#maybe changing the optimizer and loss function can help

In [ ]:
model.fit( 
    [en_in_data, dec_in_data],
    dec_tr_data, 
    batch_size=100, 
    epochs=200, 
    validation_split=0.2,
    )

Epoch 1/200
21/21 [==============================] - 16s 714ms/step - loss: 5.0921 - val_loss: 1.5014
Epoch 2/200
21/21 [==============================] - 14s 657ms/step - loss: 1.2654 - val_loss: 1.1841
Epoch 3/200
21/21 [==============================] - 14s 659ms/step - loss: 1.0517 - val_loss: 1.0383
Epoch 4/200
21/21 [==============================] - 14s 660ms/step - loss: 0.9664 - val_loss: 1.0023
Epoch 5/200
21/21 [==============================] - 14s 664ms/step - loss: 0.9362 - val_loss: 0.9830
Epoch 6/200
21/21 [==============================] - 14s 665ms/step - loss: 0.9168 - val_loss: 0.9725
Epoch 7/200
21/21 [==============================] - 14s 663ms/step - loss: 0.9039 - val_loss: 0.9653
Epoch 8/200
21/21 [==============================] - 14s 669ms/step - loss: 0.8931 - val_loss: 0.9565
Epoch 9/200
21/21 [==============================] - 14s 682ms/step - loss: 0.8812 - val_loss: 0.9460
Epoch 10/200
21/21 [==============================] - 14s 677ms/step - loss: 0.866

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import models

In [ ]:
model.save("/content/drive/MyDrive/Seq2seq_word2vec_model")
latent_dim=100
model = models.load_model("/content/drive/MyDrive/Seq2seq_word2vec_model")
en_outputs,state_h_enc,state_c_enc = model.layers[6].output
en_states=[state_h_enc,state_c_enc]
en_model = Model(model.input[0],[en_outputs]+en_states)

In [ ]:
model = models.load_model("/content/drive/MyDrive/Seq2seq_word2vec_model")
en_outputs,state_h_enc,state_c_enc = model.layers[6].output
en_states=[state_h_enc,state_c_enc]
en_model = Model(model.input[0],[en_outputs]+en_states)

In [ ]:
dec_state_input_h = Input(shape=(100,))
dec_state_input_c = Input(shape=(100,))
dec_hidden_state_input = Input(shape=(200,100))
dec_inputs = model.input[1]
dec_emb_layer = model.layers[5]
dec_lstm = model.layers[7]
dec_embedding= dec_emb_layer(dec_inputs)
dec_outputs2, state_h2, state_c2 = dec_lstm(dec_embedding, initial_state=[dec_state_input_h,dec_state_input_c])
attention = model.layers[8]
attn_out2 = attention([dec_outputs2,dec_hidden_state_input])
merge2 = Concatenate(axis=-1)([dec_outputs2, attn_out2])
dec_dense = model.layers[10]

dec_outputs2 = dec_dense(merge2)
dec_model = Model(
[dec_inputs] + [dec_hidden_state_input,dec_state_input_h,dec_state_input_c],
[dec_outputs2] + [state_h2, state_c2])

In [ ]:
import numpy as np
reverse_target_word_index = tr_tokenizer.index_word
reverse_source_word_index = in_tokenizer.index_word
target_word_index = tr_tokenizer.word_index
reverse_target_word_index[0]=' '

def decode_sequence(input_seq):
    #get the encoder output and states by passing the input sequence
    en_out, en_h, en_c= en_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['start']
    stop_condition = False
    #append every predicted word in decoded sentence
    decoded_sentence = ""
    while not stop_condition: 
        #get predicted output, hidden and cell state.
        output_words, dec_h, dec_c= dec_model.predict([target_seq] + [en_out,en_h, en_c])
        
        #get the index and from the dictionary get the word for that index.
        word_index = np.argmax(output_words[0, -1, :])
        text_word = reverse_target_word_index[word_index]
        decoded_sentence += text_word +" "
        if text_word == "end" or len(decoded_sentence) > max_len_text:
              stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = word_index
        en_h, en_c = dec_h, dec_c
  #return the decoded sentence
    return decoded_sentence

In [ ]:
list(X_test)[3]

'confluent consolidation seen lateral aspect left lung base regions scattered focal opacities present within right left lungs definite correlate seen scattered opacities prior torso ct although developing atelectasis seen left base prior ct examination cardiac mediastinal contours probably within normal limits given technique pneumothorax seen although right lateral lung within imaged field'

In [ ]:
sum1=list(y_test)[3]
print(sum1)

_START_ interval development of multifocal pneumonia _END_


In [ ]:
#inp_review = input("Enter : ")
#print(“Review :”,inp_review)

#inp_review = clean(inp_review,"inputs")

# i need some other mechanism to celan the inpout text data
#inp_review = ' '.join(inp_review)
input_review=list(X_test)[3]
inp_x= in_tokenizer.texts_to_sequences([input_review]) 
inp_x= pad_sequences(inp_x,  maxlen=max_len_text, padding='post')
 
summary=decode_sequence(inp_x.reshape(1,max_len_text))
if 'eos' in summary :
  summary=summary.replace('eos','')
print("\nPredicted summary:",summary);print("\n")


Predicted summary: unchanged appearance of the chest tube with known large right pleural effusion new retrocardiac opacity most likely representing areas of small pleural effusion unchanged left lower lobe atelectasis end 




So this is the use of rouge metric there are several different ways of using rouge metric per sentence either you can calcukate average but don't know why sentences gives rouge 2 scores zero

In [ ]:
from rouge import Rouge 
hypothesis = summary
reference = sum1
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'r': 0.14285714285714285, 'p': 0.04, 'f': 0.06249999658203144}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.14285714285714285, 'p': 0.04, 'f': 0.06249999658203144}}]
